### Notes
- Used video archives sorted by popularity (i.e., views):
    - CS:GO: https://www.twitch.tv/esl_csgo/videos?filter=archives&sort=views
- Twitch API documentation:
    - https://dev.twitch.tv/docs/api/reference#get-global-emotes

In [1]:
import subprocess
import os
import requests
import json
from glob import glob
from tqdm import tqdm

import pandas as pd
import numpy as np

from findpeaks import findpeaks

from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip, ffmpeg_extract_audio

In [2]:
# settings
downloadChat = True
downloadVideo = True
clipAnalysisDuration = 20
clipDuration = 60

peakMethod = 'peakdetect'
peakInterpolate = None
peakLookahead = 3

headTopVideos = 10
sampleVideos = 3
sampleClipsPerVideo = 2

quantileBins = 21
quantileCuts = [7-1, 14-1, 21-1]

cutOff = 3 * 10

channelName = 'esl_csgo'

clientID = '...'
clientSecret = '...'

randomSeed = 123
sampleSeed = randomSeed

In [3]:
def WriteJSON (data, fileName):
    with open(fileName, 'w') as f:
        json.dump(data, f)
        
def ReadJSON (fileName):
    with open(fileName, 'r', encoding = 'utf-8') as f:
        return(json.load(f))
        
def FetchTwitchData (request, clientID, accessToken):
    res = requests.get(request, headers = {'Client-ID': clientID, 'Authorization': 'Bearer ' + accessToken})
    resJSON = json.loads(res.text)

    return(resJSON)

def PullPaginatedVideos (userID, clientID, accessToken, limit = False):

    res = FetchTwitchData('https://api.twitch.tv/helix/videos?user_id=' + userID + '&sort=time&type=all&period=all&first=100&type=archive', clientID, accessToken)
    dat = pd.DataFrame(res['data'], index = None)
    pagination = res['pagination']['cursor']

    i = 1

    while i >= 0:

        res = FetchTwitchData('https://api.twitch.tv/helix/videos?user_id=' + userID + '&sort=time&type=all&period=all&first=100&type=archive&after=' + pagination, clientID, accessToken)
        dat = pd.concat([dat, pd.DataFrame(res['data'], index = None)]).reset_index(drop = True)
        try:
            pagination = res['pagination']['cursor']
        except:
            break

        if (i == limit):
            break

        i += 1
    
    return(dat)

In [4]:
res = requests.post('https://id.twitch.tv/oauth2/token', {'client_id': clientID, 'client_secret': clientSecret, 'grant_type': 'client_credentials'})
keys = res.json()
accessToken = keys['access_token']

res = FetchTwitchData('https://api.twitch.tv/helix/users?login=' + channelName, clientID, accessToken)
userID = res['data'][0]['id']

In [5]:
res = PullPaginatedVideos(userID, clientID, accessToken)

display(res.shape)

res.sort_values('view_count', ascending = False).head()

(3633, 17)

,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments
254,922796862,41177895292,31239503,esl_csgo,ESL_CSGO,RERUN: G2 Esports vs. Evil Geniuses [Dust2] Ma...,,2021-02-21T10:02:28Z,2021-02-21T10:02:28Z,https://www.twitch.tv/videos/922796862,https://static-cdn.jtvnw.net/cf_vods/d3c27h4od...,public,4495070,en,archive,12h37m37s,None
13,1088448549,42656860988,31239503,esl_csgo,ESL_CSGO,Full Broadcast: IEM Cologne 2021 - Quarterfina...,Intel® Extreme Masters Cologne 2021\nhttps://w...,2021-07-16T13:01:49Z,2021-07-16T13:01:49Z,https://www.twitch.tv/videos/1088448549,https://static-cdn.jtvnw.net/cf_vods/d2nvs3185...,public,3429248,en,archive,10h55m42s,None
12,1089501323,42666800972,31239503,esl_csgo,ESL_CSGO,Full Broadcast: IEM Cologne 2021 - Semifinals ...,Intel® Extreme Masters Cologne 2021\nhttps://w...,2021-07-17T13:10:03Z,2021-07-17T13:10:03Z,https://www.twitch.tv/videos/1089501323,https://static-cdn.jtvnw.net/cf_vods/d2nvs3185...,public,3159753,en,archive,7h49m25s,None
256,921422754,41166255804,31239503,esl_csgo,ESL_CSGO,LIVE: Ninjas in Pyjamas vs. Virtus.pro - IEM K...,,2021-02-20T10:01:17Z,2021-02-20T10:01:17Z,https://www.twitch.tv/videos/921422754,https://static-cdn.jtvnw.net/cf_vods/d3c27h4od...,public,3143647,en,archive,10h53m38s,None
14,1083340390,42607743612,31239503,esl_csgo,ESL_CSGO,Full Broadcast: IEM Cologne 2021 - Group Stage...,Intel® Extreme Masters Cologne 2021\nhttps://w...,2021-07-11T10:24:10Z,2021-07-11T10:24:10Z,https://www.twitch.tv/videos/1083340390,https://static-cdn.jtvnw.net/cf_vods/d2nvs3185...,public,3074634,en,archive,12h52m35s,None


In [6]:
chosenVODs = res.sort_values('view_count', ascending = False).head(headTopVideos).sample(sampleVideos, random_state = randomSeed).reset_index()
chosenVODs

,index,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments
0,14,1083340390,42607743612,31239503,esl_csgo,ESL_CSGO,Full Broadcast: IEM Cologne 2021 - Group Stage...,Intel® Extreme Masters Cologne 2021\nhttps://w...,2021-07-11T10:24:10Z,2021-07-11T10:24:10Z,https://www.twitch.tv/videos/1083340390,https://static-cdn.jtvnw.net/cf_vods/d2nvs3185...,public,3074634,en,archive,12h52m35s,None
1,254,922796862,41177895292,31239503,esl_csgo,ESL_CSGO,RERUN: G2 Esports vs. Evil Geniuses [Dust2] Ma...,,2021-02-21T10:02:28Z,2021-02-21T10:02:28Z,https://www.twitch.tv/videos/922796862,https://static-cdn.jtvnw.net/cf_vods/d3c27h4od...,public,4495070,en,archive,12h37m37s,None
2,201,983588848,41732716828,31239503,esl_csgo,ESL_CSGO,Full Broadcast: ESL Pro League Season 13 - Gra...,All about ESL Pro League Season 13\nhttps://pr...,2021-04-11T11:08:26Z,2021-04-11T11:08:26Z,https://www.twitch.tv/videos/983588848,https://static-cdn.jtvnw.net/cf_vods/d2nvs3185...,public,2596764,en,archive,10h5m27s,None


In [7]:
twitchIds = chosenVODs.id.astype('str')
twitchIds

0    1083340390
1     922796862
2     983588848
Name: id, dtype: object

In [8]:
# Global Emotes
emotesGlobal = FetchTwitchData('https://api.twitch.tv/helix/chat/emotes/global', clientID, accessToken)

# Channel Emotes
emotesChannel = FetchTwitchData('https://api.twitch.tv/helix/chat/emotes?broadcaster_id=' + userID, clientID, accessToken)

# Global Badges (https://badges.twitch.tv/v1/badges/global/display?language=en)
badgesGlobal = FetchTwitchData('https://api.twitch.tv/helix/chat/badges/global', clientID, accessToken)

# Channel Badges (https://badges.twitch.tv/v1/badges/channels/ + userID + /display)
badgesChannel = FetchTwitchData('https://api.twitch.tv/helix/chat/badges?broadcaster_id=' + userID, clientID, accessToken)

# All Emotes and Badges
emotesAll = emotesGlobal['data'] + emotesChannel['data']
badgesAll = badgesChannel['data'] + badgesGlobal['data']

# Save Badges
emotesAndBadges = {'emotesGlobal': emotesGlobal,
                   'emotesChannel': emotesChannel,
                   'emotesAll': emotesAll,
                   'badgesGlobal': badgesGlobal,
                   'badgesChannel': badgesChannel,
                   'badgesAll': badgesAll}

WriteJSON(emotesAndBadges, userID + '_badges_and_emotes.json')

In [9]:
for twitchId in twitchIds:
    
    print('Now working on:', twitchId)
    
    # fetch chat and video
    if downloadChat:
        subprocess.call(['TwitchDownloader/TwitchDownloaderCLI.exe', 
                         '--mode', 'ChatDownload', 
                         '--embed-emotes', 
                         '--id', twitchId, 
                         '--output', twitchId + '.json'])

    if downloadVideo:
        subprocess.call(['TwitchDownloader/TwitchDownloaderCLI.exe', 
                         '--mode', 'VideoDownload', 
                         '--id', twitchId, 
                         '--output', twitchId + '.mp4'])

    videoFile = glob('*' + twitchId + '*.mp4')[0]
    chatFile = glob('*' + twitchId + '*.json')[0]
    
    # create chat data set
    chatData = ReadJSON(chatFile)

    chatDat = pd.DataFrame({
        'RawTime': [comment['content_offset_seconds'] for comment in chatData['comments']],
        'User': [comment['commenter']['name'] for comment in chatData['comments']],
        'Message': [comment['message']['body'] for comment in chatData['comments']]
    })

    chatDat['Time'] = pd.to_timedelta(chatDat['RawTime'], unit = 's')
    chatDat['FloorTime'] = chatDat['Time'].dt.floor(str(clipAnalysisDuration) + 'S')
    chatDat['Time (s)'] = chatDat['Time'].dt.total_seconds()
    
    # set up data for chat volume over time
    chatVolume = pd.DataFrame({
    'Chat Volume': chatDat.FloorTime.value_counts().sort_index()
    })

    chatVolume['Time (s)'] = chatVolume.index.total_seconds()
    
    # fetch emotes
    collectEmotes = []

    for comment in chatData['comments']:
        if 'emoticons' in comment['message'].keys():
            if comment['message']['emoticons'] is not None:
                for emoticon in comment['message']['emoticons']:
                    collectEmotes.append(emoticon['_id'])

    uniqueEmotes = np.unique(collectEmotes)
    
    for eachEmote in tqdm(uniqueEmotes):
        outFile = 'emotes/' + eachEmote + '.png'
        if not os.path.isfile(outFile):
            res = requests.get('https://static-cdn.jtvnw.net/emoticons/v2/' + eachEmote + '/default/dark/1.0')
            if res.status_code is 200:
                with open(outFile, 'wb') as f:
                    f.write(res.content)
            else:
                print('Issue with emote:', eachEmote)
            
    # fetch badges
    collectBadges = []

    for comment in chatData['comments']:
        if 'user_badges' in comment['message'].keys():
            if comment['message']['user_badges'] is not None:
                for badge in comment['message']['user_badges']:
                    collectBadges.append(badge)

    collectBadgeURLs = []

    for eachBadge in collectBadges:

        keepSearching = True
        foundBadge = False

        for each in emotesAndBadges['badgesChannel']['data']:
            if eachBadge['_id'] == each['set_id']:
                for eachVersion in each['versions']:
                    if eachBadge['version'] == eachVersion['id']:
                        collectBadgeURLs.append(eachVersion['image_url_1x'])
                        keepSearching = False
                        foundBadge = True

        if keepSearching:
            for each in emotesAndBadges['badgesGlobal']['data']:
                if eachBadge['_id'] == each['set_id']:
                    for eachVersion in each['versions']:
                        if eachBadge['version'] == eachVersion['id']:
                            collectBadgeURLs.append(eachVersion['image_url_1x'])
                            foundBadge = True

        if not foundBadge:
            print('Did not find badge!')

    uniqueBadges = np.unique(collectBadgeURLs)
    
    for eachBadge in tqdm(uniqueBadges):
        fileName = eachBadge.replace('https://static-cdn.jtvnw.net/badges/v1/', '').replace('/1', '.png')
        outFile = 'badges/' + fileName
        if not os.path.isfile(outFile):
            res = requests.get(eachBadge)
            if res.status_code is 200:
                with open(outFile, 'wb') as f:
                    f.write(res.content)
            else:
                print('Issue with badge:', eachBadge)
    
    # peak analysis
    peakDat = list(chatVolume['Chat Volume'])

    fp = findpeaks(method = peakMethod,
                   interpolate = peakInterpolate,
                   lookahead = peakLookahead)

    results = fp.fit(peakDat)
    
    topPeaks = results['df'].query('peak == True').sort_values('y', ascending = False)
    
    display(topPeaks.y.quantile([.33, .66, .99]))
    
    # sample peaks by quantile
    topPeaks['quantile'] = pd.qcut(topPeaks.y, q = quantileBins)
    
    maxDiff = 0
    while maxDiff <= cutOff: # use a cutoff to ensure clips are not too close together

        LMH = topPeaks.groupby('quantile').sample(sampleClipsPerVideo, random_state = sampleSeed)
        LMH = LMH[LMH['quantile'].isin(topPeaks['quantile'].values.categories[quantileCuts])]
        LMH['Label'] = ['Low'] * sampleClipsPerVideo + ['Medium'] * sampleClipsPerVideo + ['High'] * sampleClipsPerVideo

        timeDiffs = []

        for i, x in enumerate(LMH.x):
            diffs = np.abs(LMH.x - x)
            timeDiffs.append(diffs[diffs != 0])

        maxDiff = min([x for eachList in timeDiffs for x in eachList])

        if maxDiff <= cutOff:
            print('Shuffling peak sampling due to cutoff rule!')
            sampleSeed += 1
    
    display(LMH)
    
    chatVolume['Chosen Peak'] = False
    chatVolume['Chosen Peak'].iloc[LMH.x.astype('int')] = True

    chatVolume['Label'] = False
    chatVolume['Label'].iloc[LMH.x.astype('int')] = LMH['Label']
    
    display(chatVolume.query('`Chosen Peak`'))
    
    clipPeaks = chatVolume.query('`Chosen Peak`').index
    clipPeaksSec = np.floor(clipPeaks.total_seconds()).astype('int')
    clipPeaksLabels = chatVolume.query('`Chosen Peak`').Label
    
    # set up video clips, including duration
    video = VideoFileClip(videoFile)
    display('Clip Duration: ' + str(clipDuration) + 's')
    
    # cut clip and pull clip-specific chat
    clips = {}
    clipStartTimes = []
    clipEndTimes = []
    totalChatMessages = []

    clipMargin = (clipDuration - clipAnalysisDuration)/2

    for i, eachClipPeakSec in enumerate(clipPeaksSec):
        
        # cut clip
        eachClipStartTime = eachClipPeakSec - clipMargin
        clipStartTimes.append(eachClipStartTime)
        eachClipEndTime = eachClipPeakSec - clipMargin + clipDuration
        clipEndTimes.append(eachClipEndTime)
        
        fileName = twitchId + '_highlight_' + clipPeaksLabels[i].lower() + '_' + str(int(eachClipStartTime))

        ffmpeg_extract_subclip(videoFile, eachClipStartTime, eachClipEndTime, targetname = 'temp_' + fileName + '.mp4')
        ffmpeg_extract_audio('temp_' + fileName + '.mp4', output = 'temp_audio_' + fileName + '.aac')

        clip = VideoFileClip('temp_' + fileName + '.mp4')

        clip.write_videofile(fileName + '.mp4', 
                             codec = 'libx264', 
                             audio = 'temp_audio_' + fileName + '.aac', 
                             #audio_codec = 'aac', 
                             remove_temp = False, 
                             preset = 'veryslow', 
                             verbose = False, 
                             write_logfile = False, 
                             #rewrite_audio = False, 
                             threads = None)

        clips.update({fileName + '.mp4': clip})
        clip.close()
        
        # remove temp files
        os.remove('temp_' + fileName + '.mp4')
        os.remove('temp_audio_' + fileName + '.aac')
        
        # pull clip-specific chat
        timeBeginning = str(int(clipStartTimes[i]))
        timeEnding = str(int(clipEndTimes[i]))
        chatIndex = chatDat.query('`Time (s)` >= ' + timeBeginning + ' and `Time (s)` <= ' + timeEnding).index
        totalChatMessages.append(len(chatIndex))

        print('---')
        print('Clip Type:', clipPeaksLabels[i])
        print('Time:', timeBeginning, '-', timeEnding)
        print('Message Count:', len(chatIndex))

        chatOut = {'streamer': chatData['streamer'], 
                   'video': chatData['video'], 
                   'comments': chatData['comments'][chatIndex[0]:(chatIndex[-1] + 1)]}
        WriteJSON(chatOut, twitchId + '_highlight_' + clipPeaksLabels[i].lower() + '_' + str(int(clipStartTimes[i])) + '.json')

Now working on: 1083340390


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:02<00:00, 13.88it/s]

[findpeaks] >Finding peaks in 1d-vector using [peakdetect] method..


0.33     92.36
0.66    155.00
0.99    222.00
Name: y, dtype: float64

Shuffling peak sampling due to cutoff rule!


...

Moviepy - Done !
Moviepy - video ready 983588848_highlight_medium_11880.mp4
---
Clip Type: Medium
Time: 11880 - 11940
Message Count: 390
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video 983588848_highlight_medium_23020.mp4.
Moviepy - Writing video 983588848_highlight_medium_23020.mp4



Moviepy - Done !
Moviepy - video ready 983588848_highlight_medium_23020.mp4
---
Clip Type: Medium
Time: 23020 - 23080
Message Count: 396
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video 983588848_highlight_low_24340.mp4.
Moviepy - Writing video 983588848_highlight_low_24340.mp4



Moviepy - Done !
Moviepy - video ready 983588848_highlight_low_24340.mp4
---
Clip Type: Low
Time: 24340 - 24400
Message Count: 300
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video 983588848_highlight_high_33620.mp4.
Moviepy - Writing video 983588848_highlight_high_33620.mp4



Moviepy - Done !
Moviepy - video ready 983588848_highlight_high_33620.mp4
---
Clip Type: High
Time: 33620 - 33680
Message Count: 548
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video 983588848_highlight_high_34900.mp4.
Moviepy - Writing video 983588848_highlight_high_34900.mp4



Moviepy - Done !
Moviepy - video ready 983588848_highlight_high_34900.mp4
---
Clip Type: High
Time: 34900 - 34960
Message Count: 637


In [10]:
chatVolume

,Chat Volume,Time (s),Chosen Peak,Label
0 days 00:00:00,14,0.0,False,False
0 days 00:00:20,11,20.0,False,False
0 days 00:00:40,5,40.0,False,False
0 days 00:01:00,5,60.0,False,False
0 days 00:01:20,9,80.0,False,False
...,...,...,...,...
0 days 10:04:00,23,36240.0,False,False
0 days 10:04:20,24,36260.0,False,False
0 days 10:04:40,23,36280.0,False,False
0 days 10:05:00,25,36300.0,False,False


In [11]:
highlightLinks = []
highlightChatLengths = []

for twitchId in twitchIds:
    highlightClips = glob(twitchId + '_highlight_*.mp4')
    highlightClipFiles = [clip.replace('.mp4', '') for clip in highlightClips]
    
    highlightChats = glob(twitchId + '_highlight_*.json')
    highlightChatLengths += [len(ReadJSON(eachHighlightChat)['comments']) for eachHighlightChat in highlightChats]
    
    clipData = pd.DataFrame([clip.split('_') for clip in highlightClips])
    clipData['Link'] = ['https://dobolyi.com/qualtrics/index.htm?VideoID=' + file + '&ShowChat=true&ChatRefresh=1&ChatRate=1' for file in highlightClipFiles]
        
    if len(highlightLinks) == 0:
        highlightLinks = clipData
    else:
        highlightLinks = pd.concat([highlightLinks, clipData], ignore_index = True)
        
highlightLinks.columns = ['Twitch ID', 'Type', 'Level', 'Time', 'Link']
highlightLinks.drop(columns = 'Type', inplace = True)
highlightLinks.Time = highlightLinks.Time.str.replace('.mp4', '')
highlightLinks['NumMessages'] = highlightChatLengths
highlightLinks['Notes'] = ''
highlightLinks.Level = pd.Categorical(highlightLinks.Level, categories = ['low', 'medium', 'high'], ordered = True)

highlightLinks.sort_values(['Twitch ID', 'Level']).to_csv('links.csv', index = False)

highlightLinks

,Twitch ID,Level,Time,Link,NumMessages,Notes
0,1083340390,high,11420,https://dobolyi.com/qualtrics/index.htm?VideoI...,441,
1,1083340390,high,20100,https://dobolyi.com/qualtrics/index.htm?VideoI...,536,
2,1083340390,low,22380,https://dobolyi.com/qualtrics/index.htm?VideoI...,220,
3,1083340390,low,36740,https://dobolyi.com/qualtrics/index.htm?VideoI...,195,
4,1083340390,medium,39220,https://dobolyi.com/qualtrics/index.htm?VideoI...,298,
5,1083340390,medium,4760,https://dobolyi.com/qualtrics/index.htm?VideoI...,323,
6,922796862,high,37020,https://dobolyi.com/qualtrics/index.htm?VideoI...,509,
7,922796862,high,43900,https://dobolyi.com/qualtrics/index.htm?VideoI...,587,
8,922796862,low,19840,https://dobolyi.com/qualtrics/index.htm?VideoI...,293,
9,922796862,low,5220,https://dobolyi.com/qualtrics/index.htm?VideoI...,230,
